In [1]:
#HIDDEN
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import pickle
import ipywidgets as widgets

from matplotlib import rc

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

params= {'text.latex.preamble' : [r'\usepackage{amsmath}']}
plt.rcParams.update(params)

In [2]:
#HIDDEN
def add_corner_arc(ax, line, radius=.7, color=None, text=None, text_radius=.5, text_rotatation=0, **kwargs):
    ''' display an arc for p0p1p2 angle
    Inputs:
        ax     - axis to add arc to
        line   - MATPLOTLIB line consisting of 3 points of the corner
        radius - radius to add arc
        color  - color of the arc
        text   - text to show on corner
        text_radius     - radius to add text
        text_rotatation - extra rotation for text
        kwargs - other arguments to pass to Arc
    '''

    lxy = line.get_xydata()

    if len(lxy) < 3:
        raise ValueError('at least 3 points in line must be available')

    p0 = lxy[0]
    p1 = lxy[1]
    p2 = lxy[2]

    width = np.ptp([p0[0], p1[0], p2[0]])
    height = np.ptp([p0[1], p1[1], p2[1]])

    n = np.array([width, height]) * 1.0
    p0_ = (p0 - p1) / n
    p1_ = (p1 - p1)
    p2_ = (p2 - p1) / n 

    theta0 = -get_angle(p0_, p1_)
    theta1 = -get_angle(p2_, p1_)

    if color is None:
        # Uses the color line if color parameter is not passed.
        color = line.get_color() 
    arc = ax.add_patch(mpl.patches.Arc(p1, width * radius, height * radius, 0, theta0, theta1, color=color, **kwargs))

    if text:
        v = p2_ / np.linalg.norm(p2_)
        if theta0 < 0:
            theta0 = theta0 + 360
        if theta1 < 0:
            theta1 = theta1 + 360
        theta = (theta0 - theta1) / 2 + text_rotatation
        pt = np.dot(rotation_transform(theta), v[:,None]).T * n * text_radius
        pt = pt + p1
        pt = pt.squeeze()
        ax.text(pt[0], pt[1], text,         
                horizontalalignment='left',
                verticalalignment='top',)

    return arc


def get_angle(p0, p1=np.array([0,0]), p2=None):
    ''' compute angle (in degrees) for p0p1p2 corner
    Inputs:
        p0,p1,p2 - points in the form of [x,y]
    '''
    if p2 is None:
        p2 = p1 + np.array([1, 0])
    v0 = np.array(p0) - np.array(p1)
    v1 = np.array(p2) - np.array(p1)

    angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
    return np.degrees(angle)

def rotation_transform(theta):
    ''' rotation matrix given theta
    Inputs:
        theta    - theta (in degrees)
    '''
    theta = np.radians(theta)
    A = [[np.math.cos(theta), -np.math.sin(theta)],
         [np.math.sin(theta), np.math.cos(theta)]]
    return np.array(A)

In [280]:
#HIDDEN
def f(theta, P):
    fig = plt.figure(figsize=(9, 5), dpi=150)
    gs = fig.add_gridspec(2, 2)
    ax1 = fig.add_subplot(gs[:, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, 1])
    L = 2
    l = 1
    h = 1.6
    H = 10
    x0 = - l / 2
    y0 = - h / 2
    sig = P / l
    sig_theta = round(sig * np.cos(np.deg2rad(theta))**2, 2)
    tau_xy = tau_yx = tau_theta = round(sig * np.sin(np.deg2rad(theta)) * np.cos(np.deg2rad(theta)), 2)
    tau_xy_max = round(sig * np.sin(np.deg2rad(45)) * np.cos(np.deg2rad(45)), 2)
    sig_xx = round(sig * np.sin(np.deg2rad(theta))**2, 2)
    sig_yy = round(sig * np.cos(np.deg2rad(theta))**2, 2)
    sig_max = sig
    ax1.add_patch(mpl.patches.Rectangle((-0.15, -0.15), 0.3, 0.3, zorder=350, fc='none', ec='k', lw=1.25, ls=':'))
    ax1.add_patch(mpl.patches.Rectangle((-0.15, -0.15), 0.3, 0.3, zorder=300, fc='none', ec='orangered', lw=1.25, ls='-',
                                              transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax1.transData))
    ax1.add_patch(mpl.patches.Rectangle((x0, y0), l, h, zorder=-1, fc='0.6', ec='k'))
    ax1.add_patch(mpl.patches.Rectangle((x0-0.1, h/2 + 0.01), l + 0.2, 0.38, zorder=5, color='w'))
    ax1.add_patch(mpl.patches.Rectangle((x0-0.1, -h/2 - 0.01), l + 0.1, -0.38, zorder=5, color='w'))
    # theta = 10
    xs = np.linspace(- l / 2, l / 2, 100)
    ys = np.ones(100)
    ax1.plot(xs, xs * np.tan(np.deg2rad(theta)), c='orangered', lw=2)
    ax1.arrow(0, -h/2, 0, -0.3, head_width=0.05, head_length=0.09, zorder=200, fc='k', width=0.01)
    ax1.arrow(0, h/2, 0, 0.3, head_width=0.05, head_length=0.09, zorder=200, fc='k', width=0.01)
    ax1.text(0.1, -h/2 - 0.2, f'$P = {P}$' + r' $\textrm{N}$', zorder=200)
    ax1.text(0.1, +h/2 + 0.2, f'$P = {P}$' + r' $\textrm{N}$', zorder=200)
    ax1.axhline(0, l/L/2, 1 - l/L/2, ls=':', c='k')
    ax1.set_xlim(-1, 1)
    ax1.set_ylim(-1.2, 1.2)
    ax1.add_patch(mpl.patches.Arc((0, 0), 0.65, 0.65, theta2=theta))
    ax1.text(0.44 * np.cos(np.deg2rad(theta / 2)), 0.44 * np.sin(np.deg2rad(theta / 2)), r'$\theta = $' + f'{theta:.3}°',
             bbox=dict(boxstyle="round", fc="w", alpha=0.8))

    ax1.arrow(0, 0.15, 0, 0.1, head_width=0.03, head_length=0.045, zorder=200, fc='k', width=0.005)
    ax1.arrow(0, -0.15, 0, -0.1, head_width=0.03, head_length=0.045, zorder=200, fc='k', width=0.005)
    ax1.text(0.04, 0.25, '$\sigma_y$')
    ax1.text(0.04, -0.3, '$\sigma_y$')

    ax1.set_aspect('equal')
    ax2.set_aspect('equal')
    ax2.set_xlim(-0.4, 0.4)
    ax2.set_ylim(-0.4, 0.4)
    if sig_yy / sig_max > 0.04:
        ax2.text(0.06, 0.3, r'$\sigma_y^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')
        ax2.text(0.06, -0.3, r'$\sigma_y^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')
    if sig_xx / sig_max > 0.04:
        ax2.text(0.3, -0.07, r'$\sigma_x^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')
        ax2.text(-0.3, -0.07, r'$\sigma_x^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')
    if tau_xy / tau_xy_max > 0.04:
        ax2.text(0.2, 0.12, r'$\tau_{xy}^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')
        ax2.text(-0.11, -0.21, r'$\tau_{yx}^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')
        ax2.text(-0.2, -0.11, r'$\tau_{xy}^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')
        ax2.text(0.11, 0.21, r'$\tau_{yx}^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')

    ax2.add_patch(mpl.patches.Rectangle((-0.15, -0.15), 0.3, 0.3, zorder=300, fc='none', ec='orangered', lw=1.75, ls='-',
                                              transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData))
    if theta < 0.3:
        theta = 0.2
    elif theta > 89.7:
        theta = 89.8
    else:
        pass
    ax2.arrow(0.18, -0.15 * tau_xy / tau_xy_max / 2, 0, (0.3 - 0.045) * tau_xy / tau_xy_max / 2, head_width=0.03, head_length=0.045 * tau_xy / tau_xy_max / 2, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
             transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)
    ax2.arrow(-0.18, 0.15 * tau_xy / tau_xy_max / 2, 0, (-0.3 + 0.045) * tau_xy / tau_xy_max / 2, head_width=0.03, head_length=0.045 * tau_xy / tau_xy_max / 2, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
             transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)
    ax2.arrow(-0.15* tau_xy / tau_xy_max / 2, 0.18, (0.3 - 0.045)* tau_xy / tau_xy_max / 2, 0, head_width=0.03, head_length=0.045 * tau_xy / tau_xy_max / 2, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
             transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)
    ax2.arrow(0.15* tau_xy / tau_xy_max / 2, -0.18, (-0.3 + 0.045)* tau_xy / tau_xy_max / 2, 0, head_width=0.03, head_length=0.045 * tau_xy / tau_xy_max / 2, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
             transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)
    ax2.arrow(0, 0.22, 0, 0.1 * sig_yy / sig_max, head_width=0.03, head_length=0.045 * sig_yy / sig_max, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
             transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)
    ax2.arrow(0, -0.22, 0, -0.1 * sig_yy / sig_max, head_width=0.03, head_length=0.045 * sig_yy / sig_max, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
             transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)
    ax2.arrow(0.22, 0, 0.1 * sig_xx / sig_max, 0, head_width=0.03, head_length=0.045 * sig_xx / sig_max, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
             transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)
    ax2.arrow(-0.22, 0, -0.1 * sig_xx / sig_max, 0, head_width=0.03, head_length=0.045 * sig_xx / sig_max, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
             transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)
    ax3.text(0.255, 0.95, r'$P = ' + f'{P}$' + r' $\textrm{N}$', ha='center', va='top')
    ax3.text(0.255, 0.85, r'$\sigma_p^\theta = ' + f' {sig_theta:4.4} ' + r'\ \textrm{MPa}$', ha='center', va='top', color='orangered')
    ax3.text(0.255, 0.725, r'$\tau_p^\theta = ' + f' {tau_theta:4.4} ' + r'\ \textrm{MPa}$', ha='center', va='top', color='orangered')
    ax3.text(0.255, 0.575, fr'$\mathbf{{\sigma}}_{{ij}}^\theta = \left[\matrix{{\sigma_{{x}}^\theta & \tau_{{xy}}^\theta \cr \tau_{{yx}}^\theta & \sigma_{{y}}^\theta }} \right] = \left[ \matrix{{{sig_xx:4.4} & {tau_xy:4.4} \cr {tau_yx:4.4} & {sig_yy:4.4} }} \right]$', ha='center', va='top', color='orangered')
    ax3.text(0.255, 0.3, fr'$\mathbf{{\sigma}}_{{ij}} = \left[\matrix{{\sigma_{{x}} & \tau_{{xy}} \cr \tau_{{yx}} & \sigma_{{y}} }} \right] = \left[ \matrix{{0.0 & 0.0 \cr 0.0 & 500.0 }} \right]$', ha='center', va='top')

    ax1.text(-0.925, 1.125, r'a', va='center', ha='center',
             bbox=dict(boxstyle="circle", fc="w", alpha=0.8))
    ax2.text(-0.345, 0.345, r'b', va='center', ha='center',
             bbox=dict(boxstyle="circle", fc="w", alpha=0.8))
    ax3.text(-0.182, 0.935, r'c', va='center', ha='center',
             bbox=dict(boxstyle="circle", fc="w", alpha=0.8))
    ax1.set(xticks=[], xticklabels=[], yticks=[], yticklabels=[])
    ax2.set(xticks=[], xticklabels=[], yticks=[], yticklabels=[])
    ax3.set(xticks=[], xticklabels=[], yticks=[], yticklabels=[])
    # ax1.axis('off')
    # ax2.axis('off')
    # ax3.axis('off')
    ax3.set_aspect('equal')
    ax3.set_xlim(-0.25, .75)
    plt.tight_layout()
    plt.subplots_adjust(wspace=-0.4)
    plt.show()


theta_slider = widgets.FloatSlider(description=r'$\theta$ [°]',
                                   value=45,
                                   min=0,
                                   max=90,
                                   step=0.1,
                                   readout_format='.2f')

P_slider = widgets.IntSlider(description=r'P [N]',
                                   value=500,
                                   min=1,
                                   max=500,
                                   step=1)

interactive_plot = widgets.interactive(f, theta=theta_slider, P=P_slider)

# theta_slider.layout = widgets.Layout(width='445px')


output = widgets.VBox([interactive_plot.children[-1],
                      theta_slider, P_slider])
output.layout = widgets.Layout(display='flex',
                               flex_flow='column',
                               align_items='center',
                               align_content='center',
                               justify_content='center',
                               width='100%',
                               height='100%')
output
